# Most-stable allocation under distributional constraints
### Kirill Zakharov
2021

In [21]:
from gurobipy import *
import numpy as np
import random

### Introduce quotas and dimensional information

In [235]:
# Defining Sets
n = 25
m = 5

students = [f"st {i}" for i in range(n)]
jobs = [f"comp {i}" for i in range(m)]

#quotas
upperQ = list(np.random.randint(7, 8, size=m))
lowerQ = list(np.random.randint(4, 5, size=m))

In [236]:
upperQ

[7, 7, 7, 7, 7]

In [237]:
#Students' ranks of companies
a_ranks = [random.sample(list(np.arange(m)+1), m) for i in range(n)]

#Companies' scores of students
c_score = np.array([random.sample(list(np.arange(n)+1), n) for j in range(m)]).T

In [238]:
model= Model("Assignment Model")

### Introduce deficiency variables and binary variables

In [239]:
#Defining the Variable
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = model.addVar(vtype= GRB.BINARY)
        
d = {}
for i in range(n):
    for j in range(m):
        d[i,j] = model.addVar(lb=0.0, ub=float('inf'), vtype= GRB.CONTINUOUS)

### Define objective function as sum of deficiency variables

In [240]:
#Objective Function
# model.setObjective(quicksum(a_ranks[i][j]*X[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)
model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

## First type of constraints

### Define constraints

In [241]:
#Constraint-1
for i in range(n):
     model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
#Constraint-2
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])
    
#Constraint-3
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

# for i in range(n):
#     for j in range(m):
#         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
#                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])

for i in range(n):
    for j in range(m):
        model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                        + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j]) + d[i,j] >= upperQ[j])
    
for i in range(n):
    for j in range(m):
        model.addConstr(X[i,j] >= 0)

### Solution

In [242]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 285 rows, 250 columns and 2500 nonzeros
Model fingerprint: 0x8b14831b
Variable types: 125 continuous, 125 integer (125 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 281.0000000
Presolve removed 126 rows and 1 columns
Presolve time: 0.01s
Presolved: 159 rows, 249 columns, 2369 nonzeros
Variable types: 0 continuous, 249 integer (129 binary)

Root relaxation: objective 0.000000e+00, 139 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   15  281.00000    0.00000   100%     -    0s
H    0     0                      16.0000000    0.

In [250]:
res1 = np.array(model.X[:125]).reshape((n,m))
res1

array([[ 1., -0., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  1., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  1., -0.],
       [ 1., -0., -0., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0.,  1., -0., -0., -0.]])

In [272]:
a_ranksTable = np.array(a_ranks).reshape((n,m))
a_ranksTable

array([[1, 4, 3, 5, 2],
       [2, 4, 1, 3, 5],
       [3, 2, 5, 4, 1],
       [5, 3, 4, 1, 2],
       [3, 2, 4, 1, 5],
       [5, 4, 3, 2, 1],
       [2, 3, 4, 1, 5],
       [1, 2, 5, 4, 3],
       [4, 2, 1, 5, 3],
       [4, 3, 2, 5, 1],
       [4, 3, 5, 2, 1],
       [4, 3, 2, 1, 5],
       [3, 5, 4, 1, 2],
       [2, 1, 4, 3, 5],
       [2, 5, 1, 3, 4],
       [2, 4, 5, 1, 3],
       [3, 4, 2, 5, 1],
       [4, 5, 1, 3, 2],
       [2, 5, 3, 4, 1],
       [3, 4, 2, 1, 5],
       [3, 5, 4, 1, 2],
       [1, 2, 4, 3, 5],
       [4, 1, 2, 5, 3],
       [3, 4, 1, 2, 5],
       [3, 2, 4, 5, 1]])

### Check how many students have  the choice that differs from initial one

In [273]:
checkMatch = [False for i in range(n)]
for i in range(n):
    if np.where(res1[i] == 1)[0][0] == np.where(a_ranksTable[i] == 1)[0][0]:
        checkMatch[i] = True

In [274]:
checkMatch

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False]

In [276]:
n - sum(checkMatch)

4

## Second type of constraints

In [245]:
model= Model("Assignment Model")

#Defining the Variable
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = model.addVar(vtype= GRB.BINARY)
        
d = {}
for i in range(n):
    for j in range(m):
        d[i,j] = model.addVar(lb=0.0, ub=float('inf'), vtype= GRB.CONTINUOUS)
        
model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

In [246]:
#Constraint-1
for i in range(n):
     model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
#Constraint-2
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])
    
#Constraint-3
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

# for i in range(n):
#     for j in range(m):
#         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
#                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])

for i in range(n):
    for j in range(m):
        model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                        + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j])\
                        + d[i,j]*upperQ[j] >= upperQ[j])
    
for i in range(n):
    for j in range(m):
        model.addConstr(X[i,j] >= 0)

In [247]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 285 rows, 250 columns and 2500 nonzeros
Model fingerprint: 0x2cdee70a
Variable types: 125 continuous, 125 integer (125 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 40.1428578
Presolve removed 126 rows and 1 columns
Presolve time: 0.01s
Presolved: 159 rows, 249 columns, 2369 nonzeros
Variable types: 0 continuous, 249 integer (129 binary)

Root relaxation: objective 0.000000e+00, 139 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   15   40.14286    0.00000   100%     -    0s
H    0     0                       2.2857143    0.0

In [277]:
res2 = np.array(model.X[:125]).reshape((n,m))
res2

array([[ 1., -0., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  1., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  1., -0.],
       [ 1., -0., -0., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0.,  1., -0., -0., -0.]])

In [278]:
a_ranksTable2 = np.array(a_ranks).reshape((n,m))
a_ranksTable2

array([[1, 4, 3, 5, 2],
       [2, 4, 1, 3, 5],
       [3, 2, 5, 4, 1],
       [5, 3, 4, 1, 2],
       [3, 2, 4, 1, 5],
       [5, 4, 3, 2, 1],
       [2, 3, 4, 1, 5],
       [1, 2, 5, 4, 3],
       [4, 2, 1, 5, 3],
       [4, 3, 2, 5, 1],
       [4, 3, 5, 2, 1],
       [4, 3, 2, 1, 5],
       [3, 5, 4, 1, 2],
       [2, 1, 4, 3, 5],
       [2, 5, 1, 3, 4],
       [2, 4, 5, 1, 3],
       [3, 4, 2, 5, 1],
       [4, 5, 1, 3, 2],
       [2, 5, 3, 4, 1],
       [3, 4, 2, 1, 5],
       [3, 5, 4, 1, 2],
       [1, 2, 4, 3, 5],
       [4, 1, 2, 5, 3],
       [3, 4, 1, 2, 5],
       [3, 2, 4, 5, 1]])

In [279]:
checkMatch2 = [False for i in range(n)]
for i in range(n):
    if np.where(res2[i] == 1)[0][0] == np.where(a_ranksTable2[i] == 1)[0][0]:
        checkMatch2[i] = True

In [280]:
n - sum(checkMatch2)

4